In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dataOriDf = pd.read_excel('../../data/데이터정리3.xlsx')

In [2]:
ageDataOriDf = pd.read_excel('../../data/의사연령별분포.xlsx')
ageDataOriDf.iloc[:,1:]

,총/합계,총/~29,총/30~39,총/40~49,총/50~59,총/60~,남/합계,남/~29,남/30~39,남/40~49,...,남비율/30~39,남비율/40~49,남비율/50~59,남비율/60~,여비율/합계,여비율/~29,여비율/30~39,여비율/40~49,여비율/50~59,여비율/60~
0,82685,839,23488,27826,19466,11066,64371,561,14723,22244,...,0.178061,0.269021,0.204160,0.120481,0.221491,0.003362,0.106005,0.067509,0.031263,0.013352
1,85073,1963,25100,27820,19381,10809,65989,1356,15836,22270,...,0.186146,0.261775,0.197430,0.114384,0.224325,0.007135,0.108895,0.065238,0.030386,0.012671
2,88945,3459,27682,27933,19357,10514,68330,2297,17412,22366,...,0.195761,0.251459,0.188600,0.106583,0.231772,0.013064,0.115465,0.062589,0.029029,0.011625
3,91197,5078,28738,27892,19258,10231,69677,3258,18114,22374,...,0.198625,0.245337,0.183065,0.101275,0.235973,0.019957,0.116495,0.060506,0.028104,0.010910
4,93484,6955,29528,27928,19203,9870,71066,4328,18808,22359,...,0.201190,0.239175,0.178170,0.095364,0.239806,0.028101,0.114672,0.059572,0.027245,0.010216
5,96180,8795,30729,27938,19154,9564,72818,5272,19852,22420,...,0.206405,0.233105,0.173030,0.089748,0.242899,0.036629,0.113090,0.057372,0.026118,0.009690


In [3]:
pd.set_option('display.max_row', 1200)
pd.set_option('display.max_columns', 100)

#1977년 합격자부터 2019년 합격자
dataDf = dataOriDf[['년도','의대졸합격/남', '의대졸합격/여', '의전졸합격/남', '의전졸합격/여', '불합합격/남', '불합합격/여']]
dataDf = dataDf[27:70]

npData = np.around(np.array(dataDf)[:,1:7])
npData

array([[ 932.,  155.,    0.,    0.,   94.,   17.],
       [ 963.,  154.,    0.,    0.,   70.,   12.],
       [ 953.,  156.,    0.,    0.,   95.,   15.],
       [1030.,  178.,    0.,    0.,   98.,   16.],
       [1026.,  132.,    0.,    0.,  149.,   26.],
       [1220.,  111.,    0.,    0.,   59.,    8.],
       [1187.,  150.,    0.,    0.,   61.,    6.],
       [1032.,  188.,    0.,    0.,  294.,   37.],
       [1242.,  181.,    0.,    0.,  139.,   25.],
       [1656.,  302.,    0.,    0.,   94.,   14.],
       [2241.,  304.,    0.,    0.,   46.,    8.],
       [2157.,  391.,    0.,    0.,  133.,   18.],
       [2194.,  526.,    0.,    0.,  180.,   33.],
       [1983.,  493.,    0.,    0.,  260.,   62.],
       [2195.,  535.,    0.,    0.,  187.,   47.],
       [2116.,  550.,    0.,    0.,  199.,   48.],
       [2400.,  616.,    0.,    0.,   75.,   20.],
       [2282.,  522.,    0.,    0.,   81.,   21.],
       [1602.,  354.,    0.,    0.,  692.,  158.],
       [1321.,  337.,    0.,   

# 시그모이드 변형 모델

In [4]:
# 분류모델함수 생성
# x - x값, sp - 시작점, xl - x축 조정, yl - y축 조정
def sigmoid_function(x, sp, xl, yl):
    
    result = 1 / ( yl+np.exp((x-(sp+(10/xl/2)))*xl) )
    
    for i in range(np.size(x)):
        if x[i]<sp:
           result[i]=0
        if x[i]>(sp+(10/xl)):
           result[i]=0
            
    return result

# 분류모델 확률질량함수 생성
def makeClusterModel(sp, xl, yl):
    x = np.arange(1, 101, 1)
    y = sigmoid_function(x, sp, xl, yl)
    sum = np.sum(y)
    z = y/sum
    
    return z

# 사망률 적용

In [5]:
def deathRateArray(npData):
    deathWoman = pd.read_excel('../../data/여성사망률추이.xlsx')[7:,:101].iloc.set_index(['연도'])
    deathMan = pd.read_excel('../../data/남성사망률추이.xlsx')[7:,:101].iloc.set_index(['연도'])
    deathRate = np.array([deathWoman, deathMan])
    resultData = npData - np.around(deathRate*npData)
    
    return resultData

# 시그모이드 

In [6]:
def makeArrayUseModel(tuningList):
    model1 = makeClusterModel(26,tuningList[0][0],tuningList[1][0]) # 의대 남
    model2 = makeClusterModel(26,tuningList[0][1],tuningList[1][1]) # 의대 여
    model3 = makeClusterModel(28,tuningList[0][2],tuningList[1][2]) # 의전원 남
    model4 = makeClusterModel(28,tuningList[0][3],tuningList[1][3]) # 의전원 여
    model5 = makeClusterModel(27,tuningList[0][4],tuningList[1][4]) # 재시험 남
    model6 = makeClusterModel(27,tuningList[0][5],tuningList[1][5]) # 재시험 여
    resultData =  np.array([model1, model2, model3, model4, model5, model6])
    
    return resultData

# 신규인원 생성

In [7]:
def makeNewPerson(npData, tuningList):
    oldSize = 100
    yearSize = len(npData)
    modelSize = len(npData[0])
    
    #모델 적용 배열
    applyModelArray = np.zeros((yearSize,modelSize,oldSize))

    #신규인원 배열
    newPersonArray = np.zeros((yearSize,2,oldSize))

    
    modelAry = makeArrayUseModel(tuningList)
    
    for i in range(yearSize):
        for j in range(modelSize):
            applyModelArray[i][j] = np.around(modelAry[j]*np.around(npData[i][j]))

            if j%2==0:
                newPersonArray[i][0] += applyModelArray[i][j]
            else:
                newPersonArray[i][1] += applyModelArray[i][j]
    
    resultData = newPersonArray
    
    return resultData

# 최종인원 도출

In [8]:
def shiftOld(personArray):
    dataArray = np.zeros((2,len(personArray[0])))
    dataArray[0] = np.roll(personArray[0], 1)  
    dataArray[1] = np.roll(personArray[1], 1)
    dataArray[0][0] = 0
    dataArray[1][0] = 0
    
    return dataArray

def makeResultPersonArray(newPersonArray, deathRateArray, retireeRateArray):
    #누적인원 배열
    
    resultPersonArray = np.zeros((yearSize,2,oldSize))

    for i in range(yearSize):
        if i>0:
            shiftOld(resultPersonArray[i-1])
            resultPersonArray[i] = shiftOld(resultPersonArray[i-1]) + newPersonArray[i]
        else:
            resultPersonArray[i] =  newPersonArray[i]
            
            
    return resultPersonArray

In [9]:
tuningList = [[0.7, 0.7, 0.7, 0.7, 0.7, 0.7],[0.00005, 0.00005, 0.00005, 0.00005, 0.00005, 0.00005]]

makeResultPersonArray()

newPerson = makeNewPerson(npData, tuningList)

pd.DataFrame(newPerson[0])

TypeError: makeResultPersonArray() missing 3 required positional arguments: 'newPersonArray', 'deathRateArray', and 'retireeRateArray'

# 연령별 합산

In [ ]:
def sumPeopleUseAge(npData):
    yearSize = len(npData)
    
    resultData = np.zeros(yearsize, 3, 12)
    
    for i in range(yearSize):
        for j in ragne(2):
            # 남/여 소계 계산
            resultData[i][j][0] = np.sum(npData[i][j])
            resultData[i][j][1] = np.sum(npData[i][j][0:30])
            resultData[i][j][2] = np.sum(npData[i][j][30:40])
            resultData[i][j][3] = np.sum(npData[i][j][40:50])
            resultData[i][j][4] = np.sum(npData[i][j][50:60])
            resultData[i][j][5] = np.sum(npData[i][j][60:])
            
            # 남/여 비율 계산
            for n in ragne(6):
                resultData[i][j][5+n] = resultData[i][j][n]/resultData[i][2][0]
        
        for j in ragne(6):
            # 합계 계산
            resultData[i][2][j] = resultData[i][0][j] + resultData[i][1][j]  
            # 합계 비율 계산
            resultData[i][2][6+i] = resultData[i][2][j]/resultData[i][2][0]


    return resultData

# 코스트 계산

In [ ]:
def calculateCost(npData):
    ageDataOriDf = pd.read_excel('../../data/의사연령별분포.xlsx')
    ageDataOriDf.iloc[:,1:]
    
    